In [1]:
from os import times
import pandas as pd
from io import StringIO

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import numpy as np


In [2]:
df = pd.read_csv('titanic.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
X_train["Embarked"].mode()

0    S
Name: Embarked, dtype: object

In [5]:
for column in ["Age","Embarked"]:
    if X_train["Embarked"].dtypes == 'object':
        column_mode = X_train[column].mode()[0]
        X_train[column].fillna(column_mode, inplace=True)
        X_test[column].fillna(column_mode, inplace=True)
    else:
        column_mean = X_train[column].mean()
        X_train[column].fillna(column_mean, inplace=True)
        X_test[column].fillna(column_mean, inplace=True)

for column in ["Cabin"]:
    X_train.drop(column, axis=1, inplace=True)
    X_test.drop(column, axis=1, inplace=True)

In [6]:
enc = OneHotEncoder(sparse=False)
enc.fit(X_train[['Embarked']])
X_train.reset_index(drop=True, inplace=True)
one_hot_encoding = pd.DataFrame(data=enc.transform(X_train[['Embarked']]), columns=enc.categories_[0])
df = pd.concat([X_train, one_hot_encoding], axis=1)
df

/Users/frussen/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,C,Q,S
0,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S,0.0,0.0,1.0
1,719,3,"McEvoy, Mr. Michael",male,24.0,0,0,36568,15.5000,Q,0.0,1.0,0.0
2,686,2,"Laroche, Mr. Joseph Philippe Lemercier",male,25.0,1,2,SC/Paris 2123,41.5792,C,1.0,0.0,0.0
3,74,3,"Chronopoulos, Mr. Apostolos",male,26.0,1,0,2680,14.4542,C,1.0,0.0,0.0
4,883,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,S,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,107,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,S,0.0,0.0,1.0
592,271,1,"Cairns, Mr. Alexander",male,24.0,0,0,113798,31.0000,S,0.0,0.0,1.0
593,861,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,S,0.0,0.0,1.0
594,436,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,S,0.0,0.0,1.0


In [7]:
le = LabelEncoder()
le.fit(X_train[['Sex']])
label_encoding = pd.DataFrame(data=le.transform(X_train[['Sex']]), columns=['Sex'+'_Code'])
df = pd.concat([X_train, label_encoding], axis=1)
df

/Users/frussen/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/frussen/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Sex_Code
0,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S,1
1,719,3,"McEvoy, Mr. Michael",male,24.0,0,0,36568,15.5000,Q,1
2,686,2,"Laroche, Mr. Joseph Philippe Lemercier",male,25.0,1,2,SC/Paris 2123,41.5792,C,1
3,74,3,"Chronopoulos, Mr. Apostolos",male,26.0,1,0,2680,14.4542,C,1
4,883,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,S,0
...,...,...,...,...,...,...,...,...,...,...,...
591,107,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,S,0
592,271,1,"Cairns, Mr. Alexander",male,24.0,0,0,113798,31.0000,S,1
593,861,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,S,1
594,436,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,S,0


In [8]:
le = LabelEncoder()
le.fit(X_train[['Sex']])
c = le.transform(X_train[['Sex']])

/Users/frussen/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/frussen/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [16]:
#
#
#
#
#
#
#
#